# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.

# Questão 1

In [281]:
#Carregando o dataset e transformando em um dataframe

import csv
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB

def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    return dataset

data = loadCsv('carData.csv')
Colunas = ['buying','maint','doors','persons','lug_boot','safety','Class']
data= pd.DataFrame(data,columns=Colunas)
data.head()

,buying,maint,doors,persons,lug_boot,safety,Class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [282]:
#Transformando dados categoricos em binários
Cat1 = pd.get_dummies(data["buying"])
Cat2 = pd.get_dummies(data["maint"])
Cat3 = pd.get_dummies(data["doors"])
Cat4 = pd.get_dummies(data["persons"])
Cat5 = pd.get_dummies(data["lug_boot"])
Cat6 = pd.get_dummies(data["safety"])

data.drop('buying',axis=1,inplace=True)
data['B_high']=Cat1["high"]
data['B_low']=Cat1["low"]
data['B_med']=Cat1["med"]
data['B_vhigh']=Cat1["vhigh"]

data.drop('maint',axis=1,inplace=True)
data['M_high']=Cat2["high"]
data['M_low']=Cat2["low"]
data['M_med']=Cat2["med"]
data['M_vhigh']=Cat2["vhigh"]

data.drop('doors',axis=1,inplace=True)
data['D_2']=Cat3["2"]
data['D_3']=Cat3["3"]
data['D_4']=Cat3["4"]
data['D_5more']=Cat3["5more"]

data.drop('persons',axis=1,inplace=True)
data['P_2']=Cat4["2"]
data['P_4']=Cat4["4"]
data['P_more']=Cat4["more"]

data.drop('lug_boot',axis=1,inplace=True)
data['LB_big']=Cat5["big"]
data['LB_med']=Cat5["med"]
data['LB_small']=Cat5["small"]

data.drop('safety',axis=1,inplace=True)
data['S_high']=Cat6["high"]
data['S_low']=Cat6["low"]
data['S_med']=Cat6["med"]


In [283]:
#Distribuição Normal

# Questão 2

In [ ]:

def splitDataset(dataset, splitRatio):
    trainSize = int(dataset.shape[0] * splitRatio)
    trainSet = []
    copy = dataset.copy()
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [17]:
# Separar as Classes

def SeparatedbyClass(data):
    
    #Seria melhor implementada com conjuntos(set) e loopings
    
    c1 = data[data["Class"]=="unacc"].copy()
    c1.drop("Class",axis=1,inplace=True)
    c2 = data[data["Class"]=="acc"].copy()
    c2.drop("Class",axis=1,inplace=True)
    c3 = data[data["Class"]=="good"].copy()
    c3.drop("Class",axis=1,inplace=True)
    c4 = data[data["Class"]=="vgood"].copy()
    c4.drop("Class",axis=1,inplace=True)
    
    separated = {"unacc":c1, "acc":c2, "good":c3, "vgood":c4}
    
    return separated

In [38]:
#Probabilidade de Atributo da Classe
def Prob_Atr_Classe(sep,classe,atr):
    return sep[classe][atr][sep[classe][atr]==1].shape[0]/(sep[classe].shape[0])

#Probabilidade de Atributo
def Prob_Atr(data,atr):
    return data[atr][data[atr]==1].shape[0]/(data.shape[0])

#Probabilidade da Classe
def Prob_Classe(data,classe):
    return data['Class'][data['Class']==classe].shape[0]/(data.shape[0])

In [36]:
#Tabela de Probabilidades de cada Atributo para cada Classe

def Tab_Prob(data):
    
    Tab = {}
    Sep = SeparatedbyClass(data)
    colunas = list(data.columns)
    colunas.remove('Class')
    
    for classe in list(Sep):
        Tab[classe] = {}
        for coluna in colunas:
            Tab[classe][coluna] = Prob_Atr_Classe(Sep,classe,coluna)
            
    return Tab
            

In [146]:
#As Probabilidades das Classes dada uma entrada

def Prob_Todas_Classes(Tab_Prob,InputVector):
    
    Prob_Classe = {}
    
    for classe in list(Tab_Prob):
        prob_classe = 1
        Input = InputVector.copy()
        Input.drop(labels="Class",inplace=True)
        for i,x in enumerate(Input):
            if(x==1):
                atr = Input.index[i]
                prob_classe *= Tab_Prob[classe][atr]
                
        Prob_Classe[classe] = prob_classe
    
    return Prob_Classe
                
            

In [180]:
#Função Para Predizer 1 Classe

def Predict(Tab_Prob,InputVector):
    Probs_Classes = Prob_Todas_Classes(Tab_Prob,InputVector)
    Prob_Max = max(list(Probs_Classes.values()))
    Indice_Classe = list(Probs_Classes.values()).index(Prob_Max)
    return list(Probs_Classes.keys())[Indice_Classe]
    

In [181]:
#Obter Predições para varias Instâncias
def getPredictions(Tab_Prob,testSet):
    prediction = []
    
    for i in testSet.index:
        prediction.append(Predict(Tab_Prob,testSet.loc[i,:]))
        
    return prediction

In [213]:
#Obter Acurácia
def getAccuracy(testSet, predictions):
    correct = 0
    indices = testSet.index
    cont = 0
    
    for i in indices:
        if(testSet.loc[i,"Class"]==predictions[cont]):
            correct+=1
        cont+=1
        
    return round(((correct/testSet.shape[0]) * 100.0),3)

# Questão 3

## 1) Separando Dados

In [301]:
#Separando entre dados de Teste e de Treino
X_treino , X_teste = train_test_split(data,test_size=0.33,random_state=4)

## 2) Meu Modelo

In [302]:
#Treinando o Modelo
treino = Tab_Prob(X_treino)

In [303]:
#Testando o Modelo
teste = getPredictions(treino,X_teste)

In [304]:
#Obtendo Acurácia
print("Acuracia: {} %".format(getAccuracy(X_teste,teste)))

Acuracia: 82.312 %


## 3) Modelo do Scikit Learn

In [305]:
#Split de Dados de Treino
treino = X_treino.copy()
target_treino = treino["Class"]
treino.drop("Class",axis=1,inplace=True)

#Split de Dados de Teste
teste = X_teste.copy()
target_teste = teste["Class"]
teste.drop("Class",axis=1,inplace=True)

In [306]:
#Treinando os Modelos
modelo1 = ComplementNB()
modelo2 = MultinomialNB()
modelo3 = GaussianNB()

modelo1.fit(treino,target_treino)
modelo2.fit(treino,target_treino)
modelo3.fit(treino,target_treino)

GaussianNB(priors=None, var_smoothing=1e-09)

In [307]:
#Testando o Modelo
print("ComplementNB: {} %; MultinomialNB: {} %; GaussianNB: {} %".format(round(modelo1.score(teste,target_teste)*100,3),
round(modelo2.score(teste,target_teste)*100,3),round(modelo3.score(teste,target_teste)*100,3)))


ComplementNB: 82.837 %; MultinomialNB: 86.69 %; GaussianNB: 80.736 %


## Conclusão

##### Acurácia : GaussianNB (80.736%)  <  MatheusNB(82.312%)  <  ComplementNB(82.837%)  < MultinomialNB(86.690%)